In [ ]:
%%configure -f 
{ "jars": ["wasb:///powerbi/adal4j-1.1.2.jar", "wasb:///powerbi/microsoft-spark-powerbi-connector_2.10-0.6.0.jar"] }

In [ ]:
case class Circle(name: String, radius: Double)

In [ ]:
import scala.collection.mutable.ListBuffer

val randomGenerator = scala.util.Random

val circleListBuffer: ListBuffer[Circle] = new ListBuffer

for (i <- 1 to 5) {

    circleListBuffer += new Circle(f"Circle$i", randomGenerator.nextInt(100))
}

In [ ]:
val circleDataFrame = hiveContext.createDataFrame(circleListBuffer.toList)

circleDataFrame.take(5)

In [ ]:
import scala.math
import org.apache.spark.sql.functions._

val computeCircleArea = udf((circleRadius: Double) => {
    
    math.Pi * circleRadius * circleRadius
})

In [ ]:
val updatedCircleDataFrame = circleDataFrame.withColumn("area", computeCircleArea(circleDataFrame("radius")))

In [ ]:
val powerbiCircleDataFrame = updatedCircleDataFrame.select("radius", "area")

powerbiCircleDataFrame.take(5)

In [ ]:
import com.microsoft.spark.powerbi.authentication._
import com.microsoft.spark.powerbi.models._
import com.microsoft.spark.powerbi.common._

val powerbiClientId = "***Enter PowerBI Client Id here***" 
val powerbiAccountUsername = "***Enter PowerBI Account Username here***"
val powerbiAccountPassword = "***Enter PowerBI Account Password here***"

def initializeAuthentication(): PowerBIAuthentication = {

    val powerBIAuthentication: PowerBIAuthentication = new PowerBIAuthentication(
      PowerBIURLs.Authority,
      PowerBIURLs.Resource,
      powerbiClientId,
      powerbiAccountUsername,
      powerbiAccountPassword
    )

    powerBIAuthentication
}

In [ ]:
//Initialize PowerBI Authentication

val powerbiAuthentication: PowerBIAuthentication = initializeAuthentication()

//Create PowerBI dataset and table

val powerbiTableName = "CircleRadiusAreaTable"
val powerbiDatasetName = "CircleDataset"

val circleTableColumns = Map("Radius" -> PowerBIDataTypes.Double.toString(), "Area" -> PowerBIDataTypes.Double.toString())

val circleTable = PowerBIUtils.defineTable(powerbiTableName, circleTableColumns)

val powerbiTableList = List[table](circleTable)

val powerbiDatasetDetails = PowerBIUtils.getOrCreateDataset(powerbiDatasetName, powerbiTableList,
PowerBIOptions.basicFIFO, powerbiAuthentication.getAccessToken())

println(powerbiDatasetDetails)

In [ ]:
import com.microsoft.spark.powerbi.extensions.DataFrameExtensions._

powerbiCircleDataFrame.toPowerBI(powerbiDatasetDetails, powerbiTableList.head, powerbiAuthentication)